In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
import torchvision.models as models
import matplotlib.pyplot as plt
from torch import nn
from torch.nn import functional as F

In [2]:
learning_rate = 0.001
momentum = 0.9
training_epochs = 16
batch_size = 326
torch.manual_seed(2018171013)

In [3]:
transforms_train = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

transforms_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

trainset = torchvision.datasets.ImageFolder(root="../input/chest-xray-pneumonia/chest_xray/train", transform=transforms_train)
testset = torchvision.datasets.ImageFolder(root="../input/chest-xray-pneumonia/chest_xray/test", transform=transforms_test)

In [4]:
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False)

In [5]:
print(trainset.__getitem__(0)[0].size(), trainset.__len__())
print(testset.__getitem__(0)[0].size(), testset.__len__())

print(len(trainset),len(testset))

torch.Size([3, 224, 224]) 5216
torch.Size([3, 224, 224]) 624
5216 624


In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()           
        self.layer = nn.Sequential(                                             
            nn.Conv2d(in_channels=3,out_channels=16,kernel_size=5, padding=2), # [batch_size,3,224,224] -> [batch_size,16,224,224]
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),                              # [batch_size,16,224,224] -> [batch_size,16,112,112]
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5, padding=2),# [batch_size,16,112,112] -> [batch_size,32,112,112]
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),                              # [batch_size,32,112,112] -> [batch_size,32,56,56]
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=2),# [batch_size,32,56,56] -> [batch_size,64,56,56]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)                               # [batch_size,64,56,56] -> [batch_size,64,28,28]
        )
        self.fc_layer = nn.Sequential(                                          
            nn.Linear(64*28*28,100), #Linear(64*28*28,100)                     # [batch_size,64*28*28] -> [batch_size,100]
            nn.BatchNorm1d(100),
            nn.ReLU(),
            nn.Linear(100,1)                                                   # [batch_size,100] -> [batch_size,1]
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight.data)
                m.bias.data.fill_(0)
            
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight.data)
                m.bias.data.fill_(0)
        
    def forward(self,x):
        out = self.layer(x)
        out = out.view(-1,64*28*28)
        out = self.fc_layer(out)
    
        return torch.sigmoid(out)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
Model = CNN().to(device)

In [7]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(Model.parameters(), momentum=momentum, lr=learning_rate)

total_data = len(trainset)
iteration_num = len(trainloader)

print("LEARNING STARTS! (model: My CNN model)")
print("total data is ", total_data)
print("there will be about ", iteration_num, "steps")

LEARNING STARTS! (model: My CNN model)
total data is  5216
there will be about  16 steps


In [8]:
current_accuracy = 0
for epoch in range(training_epochs):
    avg_cost = 0
    step = 0
    
    for X, Y in trainloader:
        X = X.to(device)
        Y =Y.view(-1,1)
        Y =Y.to(torch.float32)
        Y =Y.to(device)
        optimizer.zero_grad()
        hypothesis = Model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost / iteration_num
        if(step % 100 == 0):
            print("STEP [", step, "/", iteration_num, "] LOSS: ", cost.item())
        step += 1
    print('[EPOCH: {:>4}] COST = {:>.9}'.format(epoch + 1, avg_cost))
    correct = 0
    total = 0
    Model.eval()
    with torch.no_grad():
        for images, labels in testloader:
            outputs = Model(images.to(device))
            total += labels.size(0)
            correct_prediction = outputs[outputs.data>0.5]
            correct += correct_prediction.sum()

        accuracy = int(correct) / total
        print('EPOCH', epoch+1,  ' ACCURACY:', accuracy)
        if(accuracy > current_accuracy):
            print('IMPROVEMENT WAS THERE. SAVE CKPT...')
            current_accuracy = accuracy
            
print("LEARNING FINISHED! (model: my CNN model)")
print('FINAL ACCURACY: ', current_accuracy)

STEP [ 0 / 16 ] LOSS:  0.8825727701187134
[EPOCH:    1] COST = 0.477610707
EPOCH 1  ACCURACY: 0.7403846153846154
IMPROVEMENT WAS THERE. SAVE CKPT...
STEP [ 0 / 16 ] LOSS:  0.16912376880645752
[EPOCH:    2] COST = 0.145798743
EPOCH 2  ACCURACY: 0.8541666666666666
IMPROVEMENT WAS THERE. SAVE CKPT...
STEP [ 0 / 16 ] LOSS:  0.1234840452671051
[EPOCH:    3] COST = 0.105996124
EPOCH 3  ACCURACY: 0.8381410256410257
STEP [ 0 / 16 ] LOSS:  0.10516253113746643
[EPOCH:    4] COST = 0.0916662514
EPOCH 4  ACCURACY: 0.7948717948717948
STEP [ 0 / 16 ] LOSS:  0.06527509540319443
[EPOCH:    5] COST = 0.0860374868
EPOCH 5  ACCURACY: 0.8685897435897436
IMPROVEMENT WAS THERE. SAVE CKPT...
STEP [ 0 / 16 ] LOSS:  0.08134829998016357
[EPOCH:    6] COST = 0.0864955857
EPOCH 6  ACCURACY: 0.8621794871794872
STEP [ 0 / 16 ] LOSS:  0.11626847088336945
[EPOCH:    7] COST = 0.0850443691
EPOCH 7  ACCURACY: 0.8028846153846154
STEP [ 0 / 16 ] LOSS:  0.0425746887922287
[EPOCH:    8] COST = 0.0746838599
EPOCH 8  ACCURAC